In [1]:
!pip install pytorch-adapt[lightning,ignite]

     |████████████████████████████████| 137 kB 5.4 MB/s 
     |████████████████████████████████| 397 kB 20.7 MB/s 
     |████████████████████████████████| 107 kB 33.8 MB/s 
     |████████████████████████████████| 254 kB 33.6 MB/s 
     |████████████████████████████████| 527 kB 21.9 MB/s 
     |████████████████████████████████| 829 kB 35.6 MB/s 
     |████████████████████████████████| 134 kB 45.7 MB/s 
     |████████████████████████████████| 596 kB 32.5 MB/s 
     |████████████████████████████████| 952 kB 38.3 MB/s 
     |████████████████████████████████| 1.1 MB 33.3 MB/s 
     |████████████████████████████████| 271 kB 42.6 MB/s 
     |████████████████████████████████| 94 kB 521 kB/s 
     |████████████████████████████████| 144 kB 42.9 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=1e62193844f4dc4ee5a79abd258613cce113689582ebe13b73b6f3b38538ab87
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae37

### Load a toy dataset

In [2]:
import torch
from tqdm import tqdm

from pytorch_adapt.datasets import get_mnist_mnistm

# mnist is the source domain
# mnistm is the target domain
datasets = get_mnist_mnistm(["mnist"], ["mnistm"], ".", download=True)
dataloader = torch.utils.data.DataLoader(
    datasets["train"], batch_size=32, num_workers=2
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/134178716 [00:00<?, ?it/s]

100%|██████████| 68007/68007 [00:18<00:00, 3587.80it/s]


### Load toy models

In [3]:
from pytorch_adapt.models import Discriminator, mnistC, mnistG

device = torch.device("cuda")


def get_models():
    G = mnistG(pretrained=True).to(device)
    C = mnistC(pretrained=True).to(device)
    D = Discriminator(in_size=1200, h=256).to(device)
    return {"G": G, "C": C, "D": D}


def get_optimizers(models):
    G_opt = torch.optim.Adam(models["G"].parameters(), lr=0.0001)
    C_opt = torch.optim.Adam(models["C"].parameters(), lr=0.0001)
    D_opt = torch.optim.Adam(models["D"].parameters(), lr=0.0001)
    return [G_opt, C_opt, D_opt]

### Use in vanilla PyTorch

In [4]:
from pytorch_adapt.hooks import DANNHook
from pytorch_adapt.utils.common_functions import batch_to_device

models = get_models()
optimizers = get_optimizers(models)

# Assuming that models, optimizers, and dataloader are already created.
hook = DANNHook(optimizers)
for data in tqdm(dataloader):
    data = batch_to_device(data, device)
    # Optimization is done inside the hook.
    # The returned loss is for logging.
    _, loss = hook({**models, **data})

Downloading: "https://cornell.box.com/shared/static/tdx0ts24e273j7mf3r2ox7a12xh4fdfy" to /root/.cache/torch/hub/checkpoints/mnistG-68ee7945.pth


  0%|          | 0.00/161k [00:00<?, ?B/s]

Downloading: "https://cornell.box.com/shared/static/j4zrogronmievq1csulrkai7zjm27gcq" to /root/.cache/torch/hub/checkpoints/mnistC-ac7b5a13.pth


  0%|          | 0.00/1.31M [00:00<?, ?B/s]

100%|██████████| 1844/1844 [01:28<00:00, 20.85it/s]


### Build complex algorithms

In [5]:
from pytorch_adapt.hooks import MCCHook, VATHook

models = get_models()
optimizers = get_optimizers(models)

# G and C are the Generator and Classifier models
G, C = models["G"], models["C"]
misc = {"combined_model": torch.nn.Sequential(G, C)}
hook = DANNHook(optimizers, post_g=[MCCHook(), VATHook()])
for data in tqdm(dataloader):
    data = batch_to_device(data, device)
    _, loss = hook({**models, **data, **misc})

100%|██████████| 1844/1844 [02:09<00:00, 14.24it/s]


### Wrap with your favorite PyTorch framework

In [6]:
from pytorch_adapt.adapters import DANN
from pytorch_adapt.containers import Models
from pytorch_adapt.datasets import DataloaderCreator

models = get_models()
models_cont = Models(models)
adapter = DANN(models=models_cont)
dc = DataloaderCreator(num_workers=2)
dataloaders = dc(**datasets)

#### Lightning

In [7]:
import pytorch_lightning as pl

from pytorch_adapt.frameworks.lightning import Lightning

L_adapter = Lightning(adapter)
trainer = pl.Trainer(gpus=1, max_epochs=1)
trainer.fit(L_adapter, dataloaders["train"])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: /content/lightning_logs

  | Name   | Type       | Params
--------------------------------------
0 | models | ModuleDict | 756 K 
1 | misc   | ModuleDict | 0     
--------------------------------------
756 K     Trainable params
0         Non-trainable params
756 K     Total params
3.024     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

#### Ignite

In [8]:
from pytorch_adapt.frameworks.ignite import Ignite

models = get_models()
models_cont = Models(models)
adapter = DANN(models=models_cont)

trainer = Ignite(adapter)
trainer.run(datasets, dataloader_creator=dc)

[1/1843]   0%|          |it [00:00<?]

(None, None)

### Check your model's performance

In [9]:
from pytorch_adapt.validators import SNDValidator

# Random predictions as placeholder
preds = torch.randn(1000, 100)

# Assuming predictions have been collected
target_train = {"preds": preds}
validator = SNDValidator()
score = validator(target_train=target_train)

#### Lightning

In [10]:
from pytorch_adapt.frameworks.utils import filter_datasets

models = get_models()
models_cont = Models(models)
adapter = DANN(models=models_cont)
validator = SNDValidator()
dataloaders = dc(**filter_datasets(datasets, validator))
train_loader = dataloaders.pop("train")

L_adapter = Lightning(adapter, validator=validator)
trainer = pl.Trainer(gpus=1, max_epochs=1)
trainer.fit(L_adapter, train_loader, list(dataloaders.values()))

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | models | ModuleDict | 756 K 
1 | misc   | ModuleDict | 0     
--------------------------------------
756 K     Trainable params
0         Non-trainable params
756 K     Total params
3.024     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

#### Ignite

In [11]:
from pytorch_adapt.validators import ScoreHistory

models = get_models()
models_cont = Models(models)
adapter = DANN(models=models_cont)

validator = ScoreHistory(SNDValidator())
trainer = Ignite(adapter, validator=validator)
trainer.run(datasets, dataloader_creator=dc)

[1/1843]   0%|          |it [00:00<?]

[1/1844]   0%|          |it [00:00<?]

(8.411684036254883, 1)